In [35]:
import numpy as np
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
import re

def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group()) if match else float('inf')

def calculate_charges_and_positions(data_dir):
    train_xyz_files = sorted(Path(data_dir, 'atoms', 'train').glob('*.xyz'), key=lambda x: extract_number(x.stem))

    charges_list = []
    positions_list = []
    max_atoms = 23

    for xyz_file in train_xyz_files:
        atoms, positions = _read_xyz_file(xyz_file)
        print(xyz_file)
        mol = _create_molecule(atoms, positions)
        charges = _compute_charges(mol)
        
        charges_padded = np.zeros((1, max_atoms))
        positions_padded = np.zeros((1, max_atoms, 3))
        charges_padded[0, :len(charges)] = charges
        positions_padded[0, :len(positions), :] = positions

        charges_list.append(charges_padded)
        positions_list.append(positions_padded)

    charges_array = np.concatenate(charges_list, axis=0)
    positions_array = np.concatenate(positions_list, axis=0)

    return charges_array, positions_array

def _read_xyz_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()[2:]
        atoms = []
        positions = []
        for line in lines:
            parts = line.split()
            atom = parts[0]
            x = float(parts[1])
            y = float(parts[2])
            z = float(parts[3])
            atoms.append(atom)
            positions.append([x, y, z])
    return np.array(atoms), np.array(positions)

def _create_molecule(atoms, positions):
    mol = Chem.RWMol()
    conf = Chem.Conformer(len(atoms))
    
    for i, (atom, pos) in enumerate(zip(atoms, positions)):
        element = atom
        x, y, z = pos
        atom_idx = mol.AddAtom(Chem.Atom(element))
        conf.SetAtomPosition(atom_idx, (x, y, z))
    
    mol.AddConformer(conf)
    return mol

def _compute_charges(mol):
    charges = []
    for atom in mol.GetAtoms():
        charges.append(atom.GetAtomicNum())  # Numéro atomique
    return np.array(charges)

full_charges, pos = calculate_charges_and_positions("./defi-modia-2024/data")
print("Charges shape:", full_charges.shape)
print("Positions shape:", pos.shape)

defi-modia-2024/data/atoms/train/id_1.xyz
defi-modia-2024/data/atoms/train/id_2.xyz
defi-modia-2024/data/atoms/train/id_3.xyz
defi-modia-2024/data/atoms/train/id_4.xyz
defi-modia-2024/data/atoms/train/id_5.xyz
defi-modia-2024/data/atoms/train/id_6.xyz
defi-modia-2024/data/atoms/train/id_7.xyz
defi-modia-2024/data/atoms/train/id_8.xyz
defi-modia-2024/data/atoms/train/id_9.xyz
defi-modia-2024/data/atoms/train/id_10.xyz
defi-modia-2024/data/atoms/train/id_11.xyz
defi-modia-2024/data/atoms/train/id_12.xyz
defi-modia-2024/data/atoms/train/id_13.xyz
defi-modia-2024/data/atoms/train/id_14.xyz
defi-modia-2024/data/atoms/train/id_15.xyz
defi-modia-2024/data/atoms/train/id_16.xyz
defi-modia-2024/data/atoms/train/id_17.xyz
defi-modia-2024/data/atoms/train/id_18.xyz
defi-modia-2024/data/atoms/train/id_19.xyz
defi-modia-2024/data/atoms/train/id_20.xyz
defi-modia-2024/data/atoms/train/id_21.xyz
defi-modia-2024/data/atoms/train/id_22.xyz
defi-modia-2024/data/atoms/train/id_23.xyz
defi-modia-2024/data

In [36]:
full_charges[1]

array([6., 6., 7., 6., 6., 8., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 0.])

In [37]:
pos

array([[[-1.878470e+00,  4.864090e-01,  3.089740e-01],
        [-3.516160e-01,  4.048600e-01,  2.893850e-01],
        [ 1.351510e-01, -9.067880e-01,  7.887230e-01],
        ...,
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00]],

       [[-1.506548e+00,  6.498190e-01,  1.265230e-01],
        [ 3.290100e-02,  5.463680e-01,  2.241990e-01],
        [ 4.838160e-01, -2.097950e-01,  1.420295e+00],
        ...,
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00]],

       [[-1.778334e+00, -1.197525e+00, -3.147040e-01],
        [-2.915570e-01, -1.031542e+00, -6.154850e-01],
        [ 3.601980e-01,  1.313880e-01,  1.433560e-01],
        ...,
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00],
        [ 0.000000e+00

In [5]:
import numpy as np
import torch
import time
import os
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist
from kymatio.torch import HarmonicScattering3D
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D
from kymatio.scattering3d.utils \
    import generate_weighted_sum_of_gaussians
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

In [38]:
n_molecules = pos.shape[0]

In [22]:
pos.shape

(4628, 23, 3)

In [23]:
mask = full_charges <= 2
valence_charges = full_charges * mask

mask = np.logical_and(full_charges > 2, full_charges <= 10)
valence_charges += (full_charges - 2) * mask

mask = np.logical_and(full_charges > 10, full_charges <= 18)
valence_charges += (full_charges - 10) * mask

In [24]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules):
    n_atoms = np.sum(full_charges[i] != 0)
    print(n_atoms)
    pos_i = pos[i, :n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos * delta / min_dist

14
17
17
21
15
17
15
13
23
19
17
18
12
14
13
14
14
9
17
19
23
21
21
16
16
12
14
17
19
14
15
7
21
16
14
10
13
17
14
16
15
8
15
21
19
17
14
16
16
17
12
12
19
18
14
17
18
16
16
15
16
14
17
19
14
17
18
19
21
17
17
18
12
19
10
19
15
19
15
9
12
14
11
15
10
15
13
15
19
12
17
13
12
8
15
15
23
21
17
21
12
17
15
15
9
18
21
9
13
17
21
13
15
12
18
11
17
15
14
19
14
16
16
16
21
15
23
16
12
19
19
19
14
15
13
17
16
16
21
17
19
14
17
20
16
17
19
17
13
12
18
16
16
15
21
18
16
17
19
19
20
16
12
12
16
19
14
14
14
17
16
8
13
16
16
17
14
19
18
17
21
13
16
15
15
19
15
15
16
16
21
17
19
15
12
17
13
19
21
21
19
15
17
14
17
15
16
17
17
16
18
16
12
12
14
14
17
16
16
17
21
14
13
19
11
19
19
17
17
19
17
19
15
12
14
14
13
21
18
16
18
14
17
15
16
16
15
19
16
19
19
16
15
11
14
12
16
23
15
16
13
19
14
16
11
16
16
23
17
14
16
19
12
17
12
16
19
13
15
15
17
12
15
21
21
15
14
19
16
18
21
19
17
19
17
19
15
18
17
19
19
12
15
23
12
16
14
16
16
21
18
14
16
12
13
14
15
16
16
16
16
12
13
13
17
15
16
19
19
10
15
16
16
15
21
19


In [12]:
M, N, O = 192, 128, 96

grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

In [13]:
J = 2
L = 3
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

In [14]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering.to(device)

HarmonicScattering3D()

In [28]:
batch_size = 64
n_batches = int(np.ceil(n_molecules / batch_size))

In [29]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules,   "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None
for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
                    i + 1, int(((n_batches - i - 1) * dt) // 3600),
                    int((((n_batches - i - 1) * dt) // 60) % 60),
                    int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules)

    pos_batch = pos[start:end]
    full_batch = full_charges[start:end]
    val_batch = valence_charges[start:end]

    # Calculate the density map for the nuclear charges and transfer
    # to PyTorch.
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch)
    full_density_batch = full_density_batch.to(device).float()

    # Compute zeroth-order, first-order, and second-order scattering
    # coefficients of the nuclear charges.
    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch,
                                     integral_powers)
    full_scattering = scattering(full_density_batch)

    # Compute the map for valence charges.
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch)
    val_density_batch = val_density_batch.to(device).float()

    # Compute scattering coefficients for the valence charges.
    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch,
                                    integral_powers)
    val_scattering = scattering(val_density_batch)

    # Take the difference between nuclear and valence charges, then
    # compute the corresponding scattering coefficients.
    core_density_batch = full_density_batch - val_density_batch

    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch,
                                     integral_powers)
    core_scattering = scattering(core_density_batch)

    # Stack the nuclear, valence, and core coefficients into arrays
    # and append them to the output.
    batch_order_0 = torch.stack(
        (full_order_0, val_order_0, core_order_0), dim=-1)
    batch_orders_1_and_2 = torch.stack(
        (full_scattering, val_scattering, core_scattering), dim=-1)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)

Computing solid harmonic scattering coefficients of 4628 molecules from the QM7 database on GPU
sigma: 2.0, L: 3, J: 2, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [02:31:33]
Iteration 3 ETA: [02:26:54]
Iteration 4 ETA: [02:30:20]
Iteration 5 ETA: [02:26:50]
Iteration 6 ETA: [02:26:32]
Iteration 7 ETA: [02:20:30]
Iteration 8 ETA: [02:21:29]
Iteration 9 ETA: [02:16:15]
Iteration 10 ETA: [02:15:18]
Iteration 11 ETA: [02:12:48]
Iteration 12 ETA: [02:13:11]
Iteration 13 ETA: [02:10:24]
Iteration 14 ETA: [02:08:32]
Iteration 15 ETA: [02:03:09]
Iteration 16 ETA: [02:02:44]
Iteration 17 ETA: [02:03:32]
Iteration 18 ETA: [01:56:00]
Iteration 19 ETA: [01:58:39]
Iteration 20 ETA: [01:56:31]
Iteration 21 ETA: [01:51:50]
Iteration 22 ETA: [01:51:26]
Iteration 23 ETA: [01:47:08]
Iteration 24 ETA: [01:44:34]
Iteration 25 ETA: [01:46:14]
Iteration 26 ETA: [01:43:02]
Iteration 27 ETA: [01:39:58]
Iteration 28 ETA: [01:37:52]
Iteration 29 ETA: [01:35:28]
Iteration 30 ETA: [

In [30]:
order_0 = torch.cat(order_0, dim=0)
orders_1_and_2 = torch.cat(orders_1_and_2, dim=0)

order_0 = order_0.cpu().numpy()
orders_1_and_2 = orders_1_and_2.cpu().numpy()

In [41]:
import pandas as pd
df = pd.read_csv("./defi-modia-2024/data/energies/train.csv")
energy = df["energy"]

In [33]:
order_0 = order_0.reshape((n_molecules, -1))
orders_1_and_2 = orders_1_and_2.reshape((n_molecules, -1))

In [29]:
basename = 'qm7_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)

cache_dir = get_cache_dir("experiments")

filename = os.path.join("./", 'order_0_' + basename)
np.save(filename, order_0)

filename = os.path.join("./", 'orders_1_and_2' + basename)
np.save(filename, orders_1_and_2)

NameError: name 'order_0' is not defined

In [33]:
order_0 = np.load("./order_0_qm7_L_3_J_2_sigma_2.0_MNO_(192, 128, 96)_powers_[0.5, 1.0, 2.0, 3.0].npy")
orders_1_and_2 = np.load("./orders_1_and_2qm7_L_3_J_2_sigma_2.0_MNO_(192, 128, 96)_powers_[0.5, 1.0, 2.0, 3.0].npy")
scattering_coef = np.concatenate([order_0, orders_1_and_2], axis=1)

In [42]:
target = energy

In [43]:
print(n_molecules)

4628


In [44]:
n_folds = 4

P = np.random.permutation(n_molecules).reshape((n_folds, -1))

cross_val_folds = []

for i_fold in range(n_folds):
    fold = (np.concatenate(P[np.arange(n_folds) != i_fold], axis=0),
            P[i_fold])
    cross_val_folds.append(fold)

In [49]:
import pickle
alphas = 10.0 ** (-np.arange(1, 10))
best_mae = float('inf')
best_rmse = float('inf')
best_alpha = None
best_regressor = None
X = scattering_coef
y = energy

# Itération sur les différentes valeurs de alpha
for i, alpha in enumerate(alphas):
    scaler = preprocessing.StandardScaler()
    ridge = linear_model.Ridge(alpha=alpha)

    regressor = pipeline.make_pipeline(scaler, ridge)

    target_prediction = model_selection.cross_val_predict(
        regressor, X=X, y=y, cv=cross_val_folds
    )

    MAE = np.mean(np.abs(target_prediction - y))
    RMSE = np.sqrt(np.mean((target_prediction - y) ** 2))

    print(f'Ridge regression, alpha: {alpha}, MAE: {MAE}, RMSE: {RMSE}')

    # Vérifiez si le modèle actuel est le meilleur
    if MAE < best_mae:
        best_mae = MAE
        best_rmse = RMSE
        best_alpha = alpha
        best_regressor = regressor

# Sauvegarder le meilleur modèle avec pickle
if best_regressor is not None:
    best_model_data = {
        'regressor': best_regressor,
        'scaler': scaler,
        'alpha': best_alpha
    }
    best_model_filename = f'best_ridge_model_alpha_{best_alpha:.0e}.pkl'
    with open(best_model_filename, 'wb') as model_file:
        pickle.dump(best_model_data, model_file)
    print(f'Best model saved as {best_model_filename} with alpha: {best_alpha}, MAE: {best_mae}, RMSE: {best_rmse}')


/home/smussard/.conda/envs/mysepteo/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=3.4556e-08): result may not be accurate.
  overwrite_a=True).T
/home/smussard/.conda/envs/mysepteo/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=3.4126e-08): result may not be accurate.
  overwrite_a=True).T
/home/smussard/.conda/envs/mysepteo/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=3.01332e-08): result may not be accurate.
  overwrite_a=True).T
/home/smussard/.conda/envs/mysepteo/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.98936e-08): result may not be accurate.
  overwrite_a=True).T


Ridge regression, alpha: 0.1, MAE: 0.11763065682020514, RMSE: 0.18260105221595974
Ridge regression, alpha: 0.01, MAE: 0.08206948300233435, RMSE: 0.18394552045913296
Ridge regression, alpha: 0.001, MAE: 0.06623370621760849, RMSE: 0.28084650286399704
Ridge regression, alpha: 0.0001, MAE: 0.055179863549017306, RMSE: 0.32096980384166746
Ridge regression, alpha: 1e-05, MAE: 0.05001097737755472, RMSE: 0.4274050990510258
Ridge regression, alpha: 1e-06, MAE: 0.05391786975946543, RMSE: 0.6033763828195685
Ridge regression, alpha: 1e-07, MAE: 0.05596688791762484, RMSE: 0.7262584682241967
Ridge regression, alpha: 1e-08, MAE: 0.05215014737298183, RMSE: 0.6097236518171457
Ridge regression, alpha: 1e-09, MAE: 0.04577743306789267, RMSE: 0.2530606881587037
Best model saved as best_ridge_model_alpha_1e-09.pkl with alpha: 1e-09, MAE: 0.04577743306789267, RMSE: 0.2530606881587037


In [50]:
with open(best_model_filename, 'rb') as model_file:
    loaded_model_data = pickle.load(model_file)

best_regressor_loaded = loaded_model_data['regressor']
best_scaler_loaded = loaded_model_data['scaler']
best_alpha_loaded = loaded_model_data['alpha']

print(f'Loaded model with alpha: {best_alpha_loaded}')

Loaded model with alpha: 1e-09


## Test

In [20]:
import numpy as np
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
import re

def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group()) if match else float('inf')

def calculate_charges_and_positions(data_dir):
    train_xyz_files = sorted(Path(data_dir, 'atoms', 'test').glob('*.xyz'), key=lambda x: extract_number(x.stem))

    charges_list = []
    positions_list = []
    max_atoms = 23

    for xyz_file in train_xyz_files:
        atoms, positions = _read_xyz_file(xyz_file)
        print(xyz_file)
        mol = _create_molecule(atoms, positions)
        charges = _compute_charges(mol)
        
        charges_padded = np.zeros((1, max_atoms))
        positions_padded = np.zeros((1, max_atoms, 3))
        charges_padded[0, :len(charges)] = charges
        positions_padded[0, :len(positions), :] = positions

        charges_list.append(charges_padded)
        positions_list.append(positions_padded)

    charges_array = np.concatenate(charges_list, axis=0)
    positions_array = np.concatenate(positions_list, axis=0)

    return charges_array, positions_array

def _read_xyz_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()[2:]
        atoms = []
        positions = []
        for line in lines:
            parts = line.split()
            atom = parts[0]
            x = float(parts[1])
            y = float(parts[2])
            z = float(parts[3])
            atoms.append(atom)
            positions.append([x, y, z])
    return np.array(atoms), np.array(positions)

def _create_molecule(atoms, positions):
    mol = Chem.RWMol()
    conf = Chem.Conformer(len(atoms))
    
    for i, (atom, pos) in enumerate(zip(atoms, positions)):
        element = atom
        x, y, z = pos
        atom_idx = mol.AddAtom(Chem.Atom(element))
        conf.SetAtomPosition(atom_idx, (x, y, z))
    
    mol.AddConformer(conf)
    return mol

def _compute_charges(mol):
    charges = []
    for atom in mol.GetAtoms():
        charges.append(atom.GetAtomicNum())  # Numéro atomique
    return np.array(charges)

full_charges, pos = calculate_charges_and_positions("./defi-modia-2024/data")
print("Charges shape:", full_charges.shape)
print("Positions shape:", pos.shape)

defi-modia-2024/data/atoms/test/id_4629.xyz
defi-modia-2024/data/atoms/test/id_4630.xyz
defi-modia-2024/data/atoms/test/id_4631.xyz
defi-modia-2024/data/atoms/test/id_4632.xyz
defi-modia-2024/data/atoms/test/id_4633.xyz
defi-modia-2024/data/atoms/test/id_4634.xyz
defi-modia-2024/data/atoms/test/id_4635.xyz
defi-modia-2024/data/atoms/test/id_4636.xyz
defi-modia-2024/data/atoms/test/id_4637.xyz
defi-modia-2024/data/atoms/test/id_4638.xyz
defi-modia-2024/data/atoms/test/id_4639.xyz
defi-modia-2024/data/atoms/test/id_4640.xyz
defi-modia-2024/data/atoms/test/id_4641.xyz
defi-modia-2024/data/atoms/test/id_4642.xyz
defi-modia-2024/data/atoms/test/id_4643.xyz
defi-modia-2024/data/atoms/test/id_4644.xyz
defi-modia-2024/data/atoms/test/id_4645.xyz
defi-modia-2024/data/atoms/test/id_4646.xyz
defi-modia-2024/data/atoms/test/id_4647.xyz
defi-modia-2024/data/atoms/test/id_4648.xyz
defi-modia-2024/data/atoms/test/id_4649.xyz
defi-modia-2024/data/atoms/test/id_4650.xyz
defi-modia-2024/data/atoms/test/

In [21]:
n_molecules_test = pos_test.shape[0]
mask_test = full_charges_test <= 2
valence_charges_test = full_charges_test * mask_test

mask_test = np.logical_and(full_charges_test > 2, full_charges_test<= 10)
valence_charges_test += (full_charges_test - 2) * mask_test

mask_test = np.logical_and(full_charges_test > 10, full_charges_test <= 18)
valence_charges_test += (full_charges_test - 10) * mask_test

In [22]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules_test):
    n_atoms_test = np.sum(full_charges_test[i] != 0)
    print(n_atoms_test)
    pos_i_test = pos_test[i, :n_atoms_test, :]
    min_dist = min(min_dist, pdist(pos_i_test).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos_test = pos_test * delta / min_dist

17
16
18
21
19
17
14
14
12
19
17
16
15
13
17
14
21
17
15
16
18
17
16
17
17
18
16
14
15
14
19
17
17
18
15
15
15
17
12
16
17
16
16
13
9
18
15
21
12
16
6
17
15
17
17
17
17
19
16
15
17
14
17
17
17
16
15
17
19
19
21
15
18
16
17
17
12
21
18
15
15
17
14
19
15
15
14
19
16
17
13
17
12
17
16
19
12
9
21
16
16
13
14
17
11
17
19
14
16
13
17
14
21
19
21
15
14
14
16
12
16
18
19
14
13
21
11
16
17
16
19
17
15
19
15
16
17
13
19
15
10
21
20
17
17
16
14
15
14
16
19
23
13
16
15
16
15
16
17
16
12
21
17
14
16
13
16
17
12
15
19
12
17
16
18
17
21
16
18
19
14
15
13
17
23
16
19
15
17
13
16
15
12
17
14
17
17
16
19
17
16
17
14
12
19
21
21
11
21
16
14
16
19
19
9
17
21
20
13
9
17
15
16
16
19
17
14
14
18
19
14
21
15
10
19
15
18
20
21
14
17
19
14
15
14
18
14
16
15
14
13
17
16
11
14
23
13
8
9
15
21
12
21
15
12
17
13
19
19
21
18
19
16
21
15
15
18
12
21
18
18
14
19
19
16
19
19
11
13
21
14
16
19
14
17
17
23
16
14
19
18
17
14
21
17
16
15
10
16
16
17
12
15
12
16
19
14
18
15
14
19
18
16
13
15
15
19
19
14
14
16
15
14
14
21
14

In [23]:
batch_size = 64
n_batches = int(np.ceil(n_molecules_test / batch_size))

In [24]:
order_0_test, orders_1_and_2_test = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules_test,   "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None
for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
                    i + 1, int(((n_batches - i - 1) * dt) // 3600),
                    int((((n_batches - i - 1) * dt) // 60) % 60),
                    int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules_test)

    pos_batch = pos_test[start:end]
    full_batch = full_charges_test[start:end]
    val_batch = valence_charges_test[start:end]

    # Calculate the density map for the nuclear charges and transfer
    # to PyTorch.
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch)
    full_density_batch = full_density_batch.to(device).float()

    # Compute zeroth-order, first-order, and second-order scattering
    # coefficients of the nuclear charges.
    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch,
                                     integral_powers)
    full_scattering = scattering(full_density_batch)

    # Compute the map for valence charges.
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch)
    val_density_batch = val_density_batch.to(device).float()

    # Compute scattering coefficients for the valence charges.
    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch,
                                    integral_powers)
    val_scattering = scattering(val_density_batch)

    # Take the difference between nuclear and valence charges, then
    # compute the corresponding scattering coefficients.
    core_density_batch = full_density_batch - val_density_batch

    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch,
                                     integral_powers)
    core_scattering = scattering(core_density_batch)

    # Stack the nuclear, valence, and core coefficients into arrays
    # and append them to the output.
    batch_order_0 = torch.stack(
        (full_order_0, val_order_0, core_order_0), dim=-1)
    batch_orders_1_and_2 = torch.stack(
        (full_scattering, val_scattering, core_scattering), dim=-1)

    order_0_test.append(batch_order_0)
    orders_1_and_2_test.append(batch_orders_1_and_2)

Computing solid harmonic scattering coefficients of 1156 molecules from the QM7 database on GPU
sigma: 2.0, L: 3, J: 2, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:36:10]
Iteration 3 ETA: [00:34:06]
Iteration 4 ETA: [00:32:26]
Iteration 5 ETA: [00:30:01]
Iteration 6 ETA: [00:27:56]
Iteration 7 ETA: [00:25:22]
Iteration 8 ETA: [00:24:29]
Iteration 9 ETA: [00:21:34]
Iteration 10 ETA: [00:18:35]
Iteration 11 ETA: [00:17:15]
Iteration 12 ETA: [00:15:03]
Iteration 13 ETA: [00:12:24]
Iteration 14 ETA: [00:10:44]
Iteration 15 ETA: [00:08:35]
Iteration 16 ETA: [00:06:25]
Iteration 17 ETA: [00:04:18]
Iteration 18 ETA: [00:02:15]
Iteration 19 ETA: [00:00:00]


In [25]:
order_0_test = torch.cat(order_0_test, dim=0)
orders_1_and_2_test = torch.cat(orders_1_and_2_test, dim=0)

order_0_test = order_0_test.cpu().numpy()
orders_1_and_2_test = orders_1_and_2_test.cpu().numpy()

order_0_test = order_0_test.reshape((n_molecules_test, -1))
orders_1_and_2_test = orders_1_and_2_test.reshape((n_molecules_test, -1))

scattering_coef_test = np.concatenate([order_0_test, orders_1_and_2_test], axis=1)

In [52]:
scattering_coef_test = best_scaler_loaded.transform(scattering_coef_test)
predictions = best_regressor.predict(scattering_coef_test)
predictions.sort()
results = pd.DataFrame(predictions, columns=['id', 'energy'])
results.to_csv('predictions.csv', index=False)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [54]:


filename = os.path.join("./", 'order_0_test' )
np.save(filename, order_0_test)

filename = os.path.join("./", 'orders_1_and_2_test'  )
np.save(filename, orders_1_and_2_test)